In [ ]:
import mysql.connector
import random
from datetime import datetime, timedelta
from decimal import Decimal, ROUND_HALF_UP

connection = mysql.connector.connect(
    host="giniewicz.it",
    user="team19",
    password="te@mzaiq",
    database="team19"
)

cursor = connection.cursor()

# Pobranie wszystkich dostępnych id_koszty i szczegółowych kosztów z tabeli Koszty_organizacji
cursor.execute("SELECT id_koszty, Nocleg_za_noc, Przewodnik, Loty, Atrakcje, Ubezpieczenie, Transport FROM Koszty_organizacji;")
koszty_organizacji = cursor.fetchall()
ilosc_id_koszty = len(koszty_organizacji)

if ilosc_id_koszty < 1:
    print("Błąd: brak dostępnych kosztów w tabeli Koszty_organizacji!")
    exit()

# Zapytanie INSERT do tabeli Realizowane_wycieczki
insert_data_query = """
INSERT INTO Realizowane_wycieczki (id_wycieczki, id_rodzaj, cena_za_osobe, ilosc_nocy, data_rozpoczecia, data_zakonczenia, id_koszty) 
VALUES (%s, %s, %s, %s, %s, %s, %s);
"""

rodzaje_wycieczek = [
    {"id_rodzaj": 1, "dni_zakres": (10, 12)},
    {"id_rodzaj": 2, "dni_zakres": (10, 12)},
    {"id_rodzaj": 3, "dni_zakres": (10, 12)},
    {"id_rodzaj": 4, "dni_zakres": (14, 16)},
    {"id_rodzaj": 5, "dni_zakres": (14, 16)},
    {"id_rodzaj": 6, "dni_zakres": (14, 16)},
    {"id_rodzaj": 7, "dni_zakres": (14, 16)},
    {"id_rodzaj": 8, "dni_zakres": (18, 20)},
    {"id_rodzaj": 9, "dni_zakres": (18, 20)},
    {"id_rodzaj": 10, "dni_zakres": (18, 20)}
]

def losuj_date_wyjazdu():
    start_date = datetime(2022, 1, 21)
    end_date = datetime(2025, 9, 30)
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    return start_date + timedelta(days=random_days)

# Losowe przetasowanie listy kosztów, aby uniknąć kolejności
random.shuffle(koszty_organizacji)

# Wstawianie danych do tabeli Realizowane_wycieczki
for i, koszty in enumerate(koszty_organizacji):  # Iterujemy po kosztach organizacji
    id_koszty = koszty[0]
    nocleg_za_noc = koszty[1]
    przewodnik = koszty[2]
    loty = koszty[3]
    atrakcje = koszty[4]
    ubezpieczenie = koszty[5]
    transport = koszty[6]

    rodzaj = random.choice(rodzaje_wycieczek)
    ilosc_nocy = random.randint(rodzaj["dni_zakres"][0], rodzaj["dni_zakres"][1]) - 1
    data_wyjazdu = losuj_date_wyjazdu()
    data_przyjazdu = data_wyjazdu + timedelta(days=ilosc_nocy + 2)

    # Obliczanie ceny za osobę
    cena_za_osobe = (
    (nocleg_za_noc * ilosc_nocy) +
    przewodnik +
    loty +
    atrakcje +
    ubezpieczenie +
    transport
)

    # Losowy procent zwiększenia od 10% do 30%
    procent_zwiekszenia = Decimal(random.uniform(0.1, 0.3))
    cena_za_osobe *= (Decimal(1) + procent_zwiekszenia)

    # Zaokrąglanie do najbliższej liczby całkowitej
    cena_za_osobe = cena_za_osobe.quantize(Decimal('1'), rounding=ROUND_HALF_UP)

    # Wstawienie danych do tabeli
    cursor.execute(insert_data_query, (i + 1, rodzaj["id_rodzaj"], cena_za_osobe, ilosc_nocy, 
                                       data_wyjazdu.strftime('%Y-%m-%d'), data_przyjazdu.strftime('%Y-%m-%d'), id_koszty))

connection.commit()

cursor.close()
connection.close()

print("Dane zostały pomyślnie wprowadzone do tabeli Realizowane_wycieczki.")


Dane zostały pomyślnie wprowadzone do tabeli Realizowane_wycieczki.
